functools 模块应用于高阶函数，即参数或（和）返回值为其他函数的函数。 通常来说，此模块的功能适用于所有可调用对象。

https://docs.python.org/zh-cn/3/library/functools.html

In [1]:
from functools import *

## @functools.cache(user_function)
简单轻量级未绑定函数缓存。 有时称为 "memoize"。

返回值与 lru_cache(maxsize=None) 相同，创建一个查找函数参数的字典的简单包装器。 因为它不需要移出旧值，所以比带有大小限制的 lru_cache() 更小更快。

In [2]:
@cache
def factorial(n):
    return n * factorial(n-1) if n else 1

print(factorial(10))      # no previously cached result, makes 11 recursive calls
print(factorial(5))       # just looks up cached value result
print(factorial(12))      # makes two new recursive calls, the other 10 are cached

3628800
120
479001600


## @functools.cached_property(func)
将一个类方法转换为特征属性，一次性计算该特征属性的值，然后将其缓存为实例生命周期内的普通属性。 类似于 property() 但增加了缓存功能。 对于在其他情况下实际不可变的高计算资源消耗的实例特征属性来说该函数非常有用。

In [3]:
class DataSet:
    def __init__(self, sequence_of_numbers):
        self._data = tuple(sequence_of_numbers)

    @cached_property
    def stdev(self):
        return statistics.stdev(self._data)

## functools.lru_cache  
@functools.lru_cache(user_function)  
@functools.lru_cache(maxsize=128, typed=False)  
一个为函数提供缓存功能的装饰器，缓存 maxsize 组传入参数，在下次以相同参数调用时直接返回上一次的结果。用以节约高开销或I/O函数的调用时间。

In [4]:
@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)


In [5]:
[fib(n) for n in range(16)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [6]:
fib.cache_info()

CacheInfo(hits=28, misses=16, maxsize=None, currsize=16)

In [7]:
import urllib
@lru_cache(maxsize=32)
def get_pep(num):
    'Retrieve text of a Python Enhancement Proposal'
    resource = 'https://www.python.org/dev/peps/pep-%04d/' % num
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [8]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep = get_pep(n)
    print(n, len(pep))

8 118091
290 52781
308 40877
320 21105
8 118091
218 20328
320 21105
279 19799
289 30033
320 21105
9991 9


In [9]:
get_pep.cache_info()

CacheInfo(hits=3, misses=8, maxsize=32, currsize=8)

In [10]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep = get_pep(n)
    print(n, len(pep))

8 118091
290 52781
308 40877
320 21105
8 118091
218 20328
320 21105
279 19799
289 30033
320 21105
9991 9


In [11]:
get_pep.cache_info()

CacheInfo(hits=14, misses=8, maxsize=32, currsize=8)

## functools.partial(func, /, *args, **keywords)
返回一个新的 部分对象，当被调用时其行为类似于 func 附带位置参数 args 和关键字参数 keywords 被调用。 如果为调用提供了更多的参数，它们会被附加到 args。 如果提供了额外的关键字参数，它们会扩展并重载 keywords。

In [12]:
from functools import partial
basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

18

## class functools.partialmethod(func, /, *args, **keywords)
返回一个新的 partialmethod 描述器，其行为类似 partial 但它被设计用作方法定义而非直接用作可调用对象。

In [13]:
class Cell:
    def __init__(self):
        self._alive = False
    @property
    def alive(self):
        return self._alive
    
    def set_state(self, state):
        self._alive = bool(state)
    
    set_alive = partialmethod(set_state, True)
    set_dead = partialmethod(set_state, False)
    
c = Cell()
print(c.alive)
c.set_alive()
print(c.alive)
c.set_dead()
print(c.alive)

False
True
False


## functools.reduce(function, iterable[, initializer])
将两个参数的 function 从左至右积累地应用到 iterable 的条目，以便将该可迭代对象缩减为单一的值。 例如，reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) 是计算 ((((1+2)+3)+4)+5) 的值。 左边的参数 x 是积累值而右边的参数 y 则是来自 iterable 的更新值。 如果存在可选项 initializer，它会被放在参与计算的可迭代对象的条目之前，并在可迭代对象为空时作为默认值。 如果没有给出 initializer 并且 iterable 仅包含一个条目，则将返回第一项。

In [14]:
def add(x, y) :            # 两数相加
    return x + y
sum1 = reduce(add, [1,2,3,4,5])   # 计算列表和：1+2+3+4+5
sum2 = reduce(lambda x, y: x+y, [1,2,3,4,5])  # 使用 lambda 匿名函数
print(sum1)
print(sum2)
print(sum([1,2,3,4,5]))

15
15
15


## @functools.wraps(wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES)
这是一个便捷函数，用于在定义包装器函数时发起调用 update_wrapper() 作为函数装饰器。 它等价于 partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated)。 

In [15]:
def my_decorator(f):
     @wraps(f)
     def wrapper(*args, **kwds):
         print('Calling decorated function')
         return f(*args, **kwds)
     return wrapper
    
def my_decorator_1(f):
     def wrapper(*args, **kwds):
         """wrapper docstring"""
         print('Calling decorated function')
         return f(*args, **kwds)
     return wrapper

In [16]:
@my_decorator
def example():
    """Docstring"""
    print('Called example function')
    
@my_decorator_1
def example_1():
    """Docstring"""
    print('Called example function')

In [17]:
example()

Calling decorated function
Called example function


In [18]:
example.__name__

'example'

In [19]:
example.__doc__

'Docstring'

In [20]:
example_1()

Calling decorated function
Called example function


In [21]:
example_1.__name__

'wrapper'

In [22]:
example_1.__doc__

'wrapper docstring'